In [1]:
import zipfile
import os, io
import numpy as np
from glob import glob
import pickle
import matplotlib.pyplot as plt

def unpickle(file):
    """
        Read downsampled 64*64 ImageNet data. Refer to: https://patrykchrabaszcz.github.io/Imagenet32/
        Return (dict): keys: ['data', 'labels', 'mean'], 
                    One file with validation data (“val_data”) contains python dictionary with fields 'data' and 'labels' (There is no 'mean' field)
    """
    with open(file, 'rb') as fo:
        dict = pickle.load(fo)
    return dict

In [3]:
# Calculate mean value among all training data
for f in sorted(glob('/data/ljc/datasets/imagenet64/train' + '/*')):
    print(f)
    dic = unpickle(f)
    mean_val = dic['mean']
    break
mean_val /= np.float32(255)

/data/ljc/datasets/imagenet64/train/train_data_batch_1


In [4]:
print(mean_val)

[0.46454885 0.46888116 0.47197611 ... 0.39271004 0.39108653 0.38879286]


In [14]:
fname = sorted(glob('/data/ljc/datasets/imagenet64/train' + '/*'))[3]
print(fname)
dic_file = unpickle(fname)
data = dic_file['data'][12]

a = data.reshape(3,64,64)
b = transform_to_rgb(data)
print(a)
print(b)

print(np.array_equal(a, b))



/data/ljc/datasets/imagenet64/train/train_data_batch_3
[[[130 136 141 ... 150 138 128]
  [134 142 144 ... 159 145 133]
  [137 143 146 ... 158 148 135]
  ...
  [152 151 149 ... 169 167 166]
  [151 156 156 ... 170 167 165]
  [155 159 161 ... 169 165 162]]

 [[136 144 151 ... 161 147 136]
  [143 152 157 ... 168 154 140]
  [146 154 158 ... 171 160 144]
  ...
  [159 158 159 ... 171 170 168]
  [162 162 163 ... 171 168 165]
  [163 164 167 ... 172 168 165]]

 [[127 146 156 ... 170 145 126]
  [137 154 162 ... 179 158 132]
  [143 158 163 ... 182 165 140]
  ...
  [159 157 156 ... 170 168 164]
  [161 161 162 ... 171 167 166]
  [160 163 166 ... 173 169 165]]]
[[[130 136 141 ... 150 138 128]
  [134 142 144 ... 159 145 133]
  [137 143 146 ... 158 148 135]
  ...
  [152 151 149 ... 169 167 166]
  [151 156 156 ... 170 167 165]
  [155 159 161 ... 169 165 162]]

 [[136 144 151 ... 161 147 136]
  [143 152 157 ... 168 154 140]
  [146 154 158 ... 171 160 144]
  ...
  [159 158 159 ... 171 170 168]
  [162 162 

In [15]:
"""
    Read batch data and save each image to .npy files
"""

# read a line from the file and transform it into RGB image
def transform_to_rgb(x):
    # red_channel = data[:4096].reshape(64, 64)
    # green_channel = data[4096:8192].reshape(64, 64)
    # blue_channel = data[8192:].reshape(64, 64)

    # # merge channels
    # rgb_image = np.dstack((red_channel, green_channel, blue_channel))
    # return rgb_image
    
    img_size = 64
    img_size2 = img_size * img_size
    
    x = np.dstack((x[:img_size2], x[img_size2:2*img_size2], x[2*img_size2:]))
    x = x.reshape((img_size, img_size, 3)).transpose(2, 0, 1)
    return x
    
    # return data.reshape(3, 64, 64)
    
split = ['train', 'val']
sp = split[1]

folder = '/data/ljc/datasets/imagenet64/' + sp
out_folder = '/data/ljc/datasets/imagenet64/processed/' + sp
label_file = open(out_folder + '/labels.txt', 'w')

cnt = 0
for f in sorted(glob(folder + '/*')):
    print('Processing', f)
    
    dic = unpickle(f)
    
    for i in range(len(dic['data'])):
        data = dic['data'][i] / np.float32(255) - mean_val
        rgb_array = transform_to_rgb(data)       # scale to [0,1]. remove mean value, as suggested by https://patrykchrabaszcz.github.io/Imagenet32/
        np.save(f'{out_folder}/{cnt}.npy', rgb_array)
        
        label_file.write(str(dic['labels'][i]) + '\n')

        cnt += 1
        
label_file.close()

Processing /data/ljc/datasets/imagenet64/val/val_data


In [20]:
from datahelpers import *

train_dataset = ImageNet64(data_dir=config.imagenet_root + '/train')
val_dataset = ImageNet64(data_dir=config.imagenet_root + '/val')

In [33]:
min(train_dataset.labels)

1